# 01_data_cleaning.ipynb

cleaning two datasets
1. [Concrete Compressive Strength](https://archive.ics.uci.edu/dataset/165/concrete+compressive+strength)
2. [Natural fiber-recycled aggregate concrete compressive strength with web-based optimized UI](https://data.mendeley.com/datasets/7chtgrwkv3/2)

In [1]:
import pandas as pd
import numpy as np

In [5]:
# compressive concrete
df1 = pd.read_csv("../data/concrete_compressive_data.csv")
df2 = pd.read_csv("../data/dataset_natural_fibers.csv")

In [32]:
df1 = df1.rename(columns={
    'Cement (component 1)(kg in a m^3 mixture)': 'cement',
    'Blast Furnace Slag (component 2)(kg in a m^3 mixture)': 'slag',
    'Fly Ash (component 3)(kg in a m^3 mixture)': 'flyash',
    'Water  (component 4)(kg in a m^3 mixture)': 'water',
    'Superplasticizer (component 5)(kg in a m^3 mixture)': 'superplasticizer',
    'Coarse Aggregate  (component 6)(kg in a m^3 mixture)': 'coarse_agg',
    'Fine Aggregate (component 7)(kg in a m^3 mixture)': 'fine_agg',
    'Age (day)': 'age',
    'Concrete compressive strength(MPa, megapascals) ': 'compressive_strength'
})
df1.head(1)

,cement,slag,flyash,water,superplasticizer,coarse_agg,fine_agg,age,compressive_strength,scm,binder,w_b_ratio,fiber_pct,fiber_type,fiber_length,recycled_agg_pct,tensile_strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99,0.0,540.0,0.3,0.0,none,0,0.0,NaN


In [33]:
df2 = df2.rename(columns={
    'Cement (kg/m3)': 'cement',
    'Water/Binder Ratio': 'w_b_ratio',
    'Fine Aggregate (kg/m3)': 'fine_agg',
    'Coarse Aggregate (kg/m3)': 'coarse_agg',
    'Recycled course aggregate percentage (%)': 'recycled_agg_pct',
    'SCM (kg/m3)': 'scm',
    'Superplasticizer (kg/m3)': 'superplasticizer',
    'Natural  fiber (%)': 'fiber_pct',
    'Fiber Type': 'fiber_type',
    'Length (mm)': 'fiber_length',
    'Age (day)': 'age',
    'Compressive Strength (MPa)': 'compressive_strength',
    'Split Tensile Strength (MPa)': 'tensile_strength'
})
df2.head(1)

,cement,w_b_ratio,fine_agg,coarse_agg,recycled_agg_pct,scm,superplasticizer,fiber_pct,fiber_type,fiber_length,age,compressive_strength,tensile_strength,Source,slag,flyash,binder,water
0,380.0,0.4,950.0,650.0,50,0.0,8.0,1.0,Kenaf,25,7,31.37,3.68,Experimental investigation of eco-friendly fib...,0.0,0.0,380.0,152.0


In [34]:
numeric_cols = [
    'cement', 'scm', 'w_b_ratio', 'slag', 'flyash',
    'fine_agg', 'coarse_agg', 'superplasticizer',
    'fiber_pct', 'fiber_length', 'age', 'compressive_strength'
]

for col in numeric_cols:
    if col in df2.columns:
        df2[col] = pd.to_numeric(df2[col], errors='coerce')

In [35]:
# Dataset 1 doesn’t have a water/binder ratio or SCM total, 
# and dataset 2 doesn’t separate SCMs into slag/fly ash.

df1['scm'] = df1['slag'] + df1['flyash']
df1['binder'] = df1['cement'] + df1['scm']
df1['w_b_ratio'] = df1['water'] / df1['binder']
df1['fiber_pct'] = 0.0
df1['fiber_type'] = 'none'
df1['fiber_length'] = 0
df1['recycled_agg_pct'] = 0.0
df1['tensile_strength'] = np.nan

df1.head(1)

,cement,slag,flyash,water,superplasticizer,coarse_agg,fine_agg,age,compressive_strength,scm,binder,w_b_ratio,fiber_pct,fiber_type,fiber_length,recycled_agg_pct,tensile_strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99,0.0,540.0,0.3,0.0,none,0,0.0,NaN


In [36]:
# Fill missing SCMs, fibers, or others if needed
df2['slag'] = 0.0
df2['flyash'] = 0.0
df2['binder'] = df2['cement'] + df2['scm']
df2['water'] = df2['w_b_ratio'] * df2['binder']

In [37]:
common_cols = [
    'cement', 'slag', 'flyash', 'scm', 'water', 'superplasticizer',
    'coarse_agg', 'fine_agg', 'binder', 'w_b_ratio',
    'recycled_agg_pct', 'fiber_pct', 'fiber_type', 'fiber_length',
    'age', 'compressive_strength', 'tensile_strength'
]

df1 = df1[common_cols]
df2 = df2[common_cols]


In [38]:
combined_df = pd.concat([df1, df2], ignore_index=True)
combined_df = combined_df.fillna(0)

print(combined_df.head())
print(combined_df.info())

   cement   slag  flyash    scm  water  superplasticizer  coarse_agg   
0   540.0    0.0     0.0    0.0  162.0               2.5      1040.0  \
1   540.0    0.0     0.0    0.0  162.0               2.5      1055.0   
2   332.5  142.5     0.0  142.5  228.0               0.0       932.0   
3   332.5  142.5     0.0  142.5  228.0               0.0       932.0   
4   198.6  132.4     0.0  132.4  192.0               0.0       978.4   

   fine_agg  binder  w_b_ratio  recycled_agg_pct  fiber_pct fiber_type   
0     676.0   540.0    0.30000               0.0        0.0       none  \
1     676.0   540.0    0.30000               0.0        0.0       none   
2     594.0   475.0    0.48000               0.0        0.0       none   
3     594.0   475.0    0.48000               0.0        0.0       none   
4     825.5   331.0    0.58006               0.0        0.0       none   

   fiber_length  age  compressive_strength tensile_strength  
0             0   28                 79.99                0 

In [68]:
combined_df['scm_frac'] = combined_df['scm'].div(combined_df['binder']).fillna(0)
combined_df['fiber_flag'] = np.where(combined_df['fiber_pct'] > 0, 1, 0)
combined_df['total_aggregate'] = combined_df['fine_agg'] + combined_df['coarse_agg']
combined_df['agg_to_binder_ratio'] = (
    combined_df['total_aggregate']
    .div(combined_df['binder'])
    .fillna(0)
)

In [69]:
combined_df.isnull().any()

cement                  False
slag                    False
flyash                  False
scm                     False
water                   False
superplasticizer        False
coarse_agg              False
fine_agg                False
binder                  False
w_b_ratio               False
recycled_agg_pct        False
fiber_pct               False
fiber_type              False
fiber_length            False
age                     False
compressive_strength    False
tensile_strength        False
scm_frac                False
fiber_flag              False
total_aggregate         False
agg_to_binder_ratio     False
dtype: bool

In [70]:
combined_df.to_csv('../data/cleaned_concrete_data.csv', index=False)